**Connect To Elasticserver**

In [4]:
from pprint import pprint 
from elasticsearch import Elasticsearch 

es = Elasticsearch("http://localhost:9200")

client_info = es.info()
print("Connected to Elasticsearch")
pprint(client_info.body)

Connected to Elasticsearch
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'T1HeaWnRTOqX_BBgREVVbA',
 'name': '64c49e436740',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-10-21T10:06:21.288851013Z',
             'build_flavor': 'default',
             'build_hash': '25d88452371273dd27356c98598287b669a03eae',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.1',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.0'}}


**Create Index with auto mappings**

In [5]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(index="my_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

We are using the dummy_data.json file to insert multiple documents in the index.

In [6]:
import json 
from tqdm import tqdm # tqdm = taqaddum (arabic word means progress)

documents_ids = []
documents = json.load(open("data.json"))

for document in tqdm(documents, total=len(documents)):
    response = es.index(index="my_index", body=document) 
    documents_ids.append(response["_id"])


100%|██████████| 5/5 [00:00<00:00,  5.92it/s]


In [7]:
documents_ids

['TB0fPJoBZCw2coN1dLIL',
 'TR0fPJoBZCw2coN1drIp',
 'Th0fPJoBZCw2coN1drJ2',
 'Tx0fPJoBZCw2coN1drLH',
 'UB0fPJoBZCw2coN1d7IB']

Count documents

In [9]:
response = es.count(index="my_index")
response

ObjectApiResponse({'count': 5, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [10]:
pprint(response.body)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'count': 5}


In [11]:
print(response["count"])

5


In [12]:
documents

[{'title': 'Sample Title 1',
  'text': 'This is the first sample document text.',
  'created_on': '2024-09-22'},
 {'title': 'Sample Title 2',
  'text': 'Elasticsearch makes searching and analyzing large amounts of data fast and efficient.',
  'created_on': '2024-10-05'},
 {'title': 'Sample Title 3',
  'text': 'Django Rest Framework simplifies API development with powerful serialization tools.',
  'created_on': '2024-11-14'},
 {'title': 'Sample Title 4',
  'text': 'Python provides a wide range of libraries for data analysis, automation, and backend development.',
  'created_on': '2025-01-10'},
 {'title': 'Sample Title 5',
  'text': 'FastAPI is an excellent choice for building high-performance APIs with async capabilities.',
  'created_on': '2025-02-02'}]

In [16]:
query = {
    "range": {
        "created_on": {
            "gte": "2024-09-22",
            "lte": "2024-10-05",
            "format":"yyyy-MM-dd"
        }
    }
}

response = es.count(index="my_index", query=query)
count = response["count"]

print(f"Total {count} result found!")

Total 2 result found!


In [17]:
es.close()